# DATA ANALYSIS - AMAZON REVIEWS
____
by Vitor Flisch Cavalanti<br>
May 2021

<b>Case study Sr. Business Analyst</b>

In [1]:
# importing libraries
import pandas as pd
import numpy as np 
import ast

<hr><h2>1. Working with Metadata file</h2><hr>

In [2]:
metadata = pd.read_csv("../files/metadata.csv", nrows=5)

In [3]:
metadata.head()

,Unnamed: 0,asin,salesRank,imUrl,categories,title,description,price,related,brand
0,0,1048791,{'Books': 6334800},http://ecx.images-amazon.com/images/I/51MKP0T4...,[['Books']],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,NaN,NaN
1,1,143561,{'Movies & TV': 376041},http://g-ecx.images-amazon.com/images/G/01/x-s...,"[['Movies & TV', 'Movies']]","Everyday Italian (with Giada de Laurentiis), V...","3Pack DVD set - Italian Classics, Parties and ...",12.99,"{'also_viewed': ['B0036FO6SI', 'B000KL8ODE', '...",NaN
2,2,37214,{'Clothing': 1233557},http://ecx.images-amazon.com/images/I/31mCncNu...,"[['Clothing, Shoes & Jewelry', 'Girls'], ['Clo...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,NaN,6.99,"{'also_viewed': ['B00JO8II76', 'B00DGN4R1Q', '...",Big Dreams
3,3,32069,NaN,http://ecx.images-amazon.com/images/I/51EzU6qu...,"[['Sports & Outdoors', 'Other Sports', 'Dance'...",Adult Ballet Tutu Cheetah Pink,NaN,7.89,"{'also_bought': ['0000032050', 'B00D0DJAEG', '...",BubuBibi
4,4,31909,{'Toys & Games': 201847},http://ecx.images-amazon.com/images/I/41xBoP0F...,"[['Sports & Outdoors', 'Other Sports', 'Dance']]",Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,7.00,"{'also_bought': ['B002BZX8Z6', 'B00JHONN1S', '...",Unknown


In [4]:
metadata = pd.read_csv("../files/metadata.csv", usecols = ['asin','salesRank','categories','title','related','brand','price'])

In [5]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9430088 entries, 0 to 9430087
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   asin        object 
 1   salesRank   object 
 2   categories  object 
 3   title       object 
 4   price       float64
 5   related     object 
 6   brand       object 
dtypes: float64(1), object(6)
memory usage: 503.6+ MB


In [6]:
# checking nulls
metadata.isnull().sum() 

asin                0
salesRank     2453361
categories      75256
title         1432740
price         3366880
related       2738415
brand         7702894
dtype: int64

In [7]:
metadata.shape

(9430088, 7)

<hr><h4>a. Formatting Sales Rank</h4><hr>

In [8]:
# Filling Null values at the Sales Rank column
metadata.salesRank.fillna("{'Null': 0}", inplace=True)

In [9]:
# replacing empty sales rank with {'Null': 0}
metadata['salesRank'] = metadata['salesRank'].replace(["{}"],"{'Null': 0}")

In [10]:
# converting sales rank to dict
metadata["salesRank"] = metadata.salesRank.apply(lambda x: ast.literal_eval(x))

In [11]:
# Getting category from sales rank category 
metadata["salesrank_category"] = metadata.salesRank.apply(lambda x: list(x.keys())[0]) 

In [12]:
metadata["salesrank_value"] = metadata.salesRank.apply(lambda x: list(x.values())[0]) 

In [13]:
# top 5 categories with more products
metadata['salesrank_category'].value_counts().head(5) 

Null                     2480856
Books                    1891411
Clothing                  712239
Music                     508134
Sports &amp; Outdoors     458489
Name: salesrank_category, dtype: int64

In [15]:
# Checking Duplicated Products
duplicateRows = metadata[metadata.duplicated(['asin'])] 
duplicateRows['asin'].count() # checking dupplicated products

0

In [16]:
del duplicateRows

In [17]:
#drop sales rank column
metadata.drop(['salesRank'], axis='columns',inplace=True)

<hr><h4>b. Exploding Categories</h4><hr>

In [19]:
metadata_cat = metadata[~metadata['categories'].isna()]
metadata_cat

,asin,categories,title,price,related,brand,salesrank_category,salesrank_value
0,0001048791,[['Books']],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,NaN,Books,6334800
1,0000143561,"[['Movies & TV', 'Movies']]","Everyday Italian (with Giada de Laurentiis), V...",12.99,"{'also_viewed': ['B0036FO6SI', 'B000KL8ODE', '...",NaN,Movies & TV,376041
2,0000037214,"[['Clothing, Shoes & Jewelry', 'Girls'], ['Clo...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,6.99,"{'also_viewed': ['B00JO8II76', 'B00DGN4R1Q', '...",Big Dreams,Clothing,1233557
3,0000032069,"[['Sports & Outdoors', 'Other Sports', 'Dance'...",Adult Ballet Tutu Cheetah Pink,7.89,"{'also_bought': ['0000032050', 'B00D0DJAEG', '...",BubuBibi,Null,0
4,0000031909,"[['Sports & Outdoors', 'Other Sports', 'Dance']]",Girls Ballet Tutu Neon Pink,7.00,"{'also_bought': ['B002BZX8Z6', 'B00JHONN1S', '...",Unknown,Toys & Games,201847
...,...,...,...,...,...,...,...,...
9430083,B00LZFHL7Y,"[['Books', 'Literature & Fiction', 'Erotica'],...",NaN,0.99,"{'also_bought': ['B00LXK4KIO', 'B00LZ19FE0', '...",NaN,Null,0
9430084,B00LV8M74W,"[['Books', 'Literature & Fiction', 'Genre Fict...",NaN,3.99,"{'also_bought': ['B00LYOC1R6', 'B00L0WXZIG', '...",NaN,Null,0
9430085,B00LXQXLGU,[['Amazon Instant Video']],NaN,NaN,NaN,NaN,Null,0
9430086,B00LV8STE4,"[['Books', 'Arts & Photography', 'Music', 'Mus...",NaN,NaN,NaN,NaN,Null,0


In [1]:
# converting categories to list
metadata_cat['categories'] = metadata_cat.categories.apply(eval)

In [21]:
# create auxiliar data frame to explode categories
metadata_categories = (metadata_cat[["asin","categories"]])

In [22]:
# exploding categories by product
metadata_categories = metadata_categories.explode("categories").explode("categories")

In [23]:
# getting the most common categories by product
group_metadata_cat = metadata_categories.groupby(['asin', 'categories']).size().reset_index(name="Count")

In [24]:
# filter the categorie(s) with most occurrences 
filter_max_cat = group_metadata_cat[group_metadata_cat.groupby(["asin"])['Count'].transform(max) == group_metadata_cat["Count"]]

In [25]:
# get the first category in case of having more than 1
filter_first_cat = filter_max_cat.groupby("asin").first().reset_index()

In [26]:
#cleaning unused data frames
del metadata_categories
del group_metadata_cat
del filter_max_cat
del metadata_cat

In [27]:
# merging with the main dataframe
metadata_join = pd.merge(metadata,filter_first_cat,on="asin")

In [29]:
# cleaning unused columns and the old dataframe
metadata_join.drop(['categories_x','Count'], axis='columns',inplace=True)
metadata_join.rename(columns = {'categories_y': 'main_category'}, inplace = True)
del metadata

<hr><h4>c. Extracting and Exploding Bought-Together</h4><hr>

In [31]:
metadata_join['related'].apply(type)

0          <class 'float'>
1            <class 'str'>
2            <class 'str'>
3            <class 'str'>
4            <class 'str'>
                ...       
9354827      <class 'str'>
9354828      <class 'str'>
9354829    <class 'float'>
9354830    <class 'float'>
9354831      <class 'str'>
Name: related, Length: 9354832, dtype: object

In [32]:
# remove products with nulls at related column
metadata_related = metadata_join[~metadata_join["related"].isna()]

In [33]:
 # remove products without bought_together
 metadata_related = metadata_related[metadata_related['related'].str.contains("bought_together")]

In [35]:
metadata_related.drop(['title','price','brand','salesrank_category','salesrank_value','main_category'], axis='columns',inplace=True)

In [36]:
# convert related column to dict
metadata_related["related"] = metadata_related.related.apply(lambda x: ast.literal_eval(x))

In [37]:
# creating new column only for bought together
metadata_related["bought_together"] = metadata_related.related.apply(lambda x: list(x.get("bought_together"))) 

In [38]:
# deleting related column and merging back to main dataframe
metadata_related.drop(['related'], axis='columns',inplace=True)

In [39]:
# merging back to the main dataframe and deleting the related column
# metadata_join = pd.merge(metadata_join,metadata_related,on="asin")

In [40]:
metadata_join

,asin,title,price,related,brand,salesrank_category,salesrank_value,main_category
0,0001048791,"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,NaN,Books,6334800,Books
1,0000143561,"Everyday Italian (with Giada de Laurentiis), V...",12.99,"{'also_viewed': ['B0036FO6SI', 'B000KL8ODE', '...",NaN,Movies & TV,376041,Movies
2,0000037214,Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,6.99,"{'also_viewed': ['B00JO8II76', 'B00DGN4R1Q', '...",Big Dreams,Clothing,1233557,"Clothing, Shoes & Jewelry"
3,0000032069,Adult Ballet Tutu Cheetah Pink,7.89,"{'also_bought': ['0000032050', 'B00D0DJAEG', '...",BubuBibi,Null,0,Clothing
4,0000031909,Girls Ballet Tutu Neon Pink,7.00,"{'also_bought': ['B002BZX8Z6', 'B00JHONN1S', '...",Unknown,Toys & Games,201847,Dance
...,...,...,...,...,...,...,...,...
9354827,B00LZFHL7Y,NaN,0.99,"{'also_bought': ['B00LXK4KIO', 'B00LZ19FE0', '...",NaN,Null,0,Erotica
9354828,B00LV8M74W,NaN,3.99,"{'also_bought': ['B00LYOC1R6', 'B00L0WXZIG', '...",NaN,Null,0,Books
9354829,B00LXQXLGU,NaN,NaN,NaN,NaN,Null,0,Amazon Instant Video
9354830,B00LV8STE4,NaN,NaN,NaN,NaN,Null,0,Arts & Photography


In [41]:
# exploding bought together
metadata_related = metadata_related.explode("bought_together")

In [42]:
metadata_related

,asin,bought_together
3,0000032069,0000032050
3,0000032069,B00D0DJAEG
3,0000032069,0000032042
3,0000032069,B00D0F450I
4,0000031909,B002BZX8Z6
...,...,...
9354657,B00LVHLRZ8,B00GJYQCB2
9354657,B00LVHLRZ8,B00K7KT2E8
9354658,B00LWWANMU,B00ER1OL08
9354658,B00LWWANMU,B00EMDL9Y2


In [44]:
metadata_join.drop(["related"], axis="columns", inplace=True)

In [45]:
metadata_join.to_csv('../exports/metadata_full.csv')

In [46]:
metadata_join.shape

(9354832, 7)

In [47]:
metadata_related.to_csv('../exports/metadata_related_full.csv')